# LASSO (coordinate descent)

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Nan\AppData\Local\Temp\graphlab_server_1529606132.log.0


This non-commercial license of GraphLab Create for academic use is assigned to nanlee_89@yahoo.com and will expire on December 07, 2018.


### Function to convert Sframe into Numpy 

In [3]:
import numpy as np 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1  # add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # combine two lists
    
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    #convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    
    # convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    
    return(feature_matrix, output_array)

### Function to predict output given the feature matrix and the weights

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions =  np.dot(feature_matrix, weights)
    return(predictions)

### Function to normalize the feature matrix.

In [6]:
def normalize_features(feature_matrix):
    # function normalizes columns of a given feature matrix 
    # The function returns a pair (normalized_features, norms), where the second item contains the norms of original features
    
    norms = np.linalg.norm(feature_matrix, axis=0)
    features = feature_matrix/norms
    return features, norms

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable. Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

We use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

### First, consider a simple model with 2 features:

In [7]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [8]:
# normalize features:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [9]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [10]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

In [11]:
ro = np.array([0,0,0])
ro[1] = np.sum(simple_feature_matrix[:,1] * (output - prediction + weights[1]*simple_feature_matrix[:,1]))
ro[2] = np.sum(simple_feature_matrix[:,2] * (output - prediction + weights[2]*simple_feature_matrix[:,2]))

print ro

[       0 87939470 80966698]


** The range of values of `l1_penalty` ***would not*** set `w[1]` zero, but ***would*** set `w[2]` to zero**

Simply calculate the value for lambda that will set w2 to zero

i.e. a lambda value that is greater than 2*ro[2]

but that is less than the value that will set w1 to zero

i.e. a lambda value that is less then 2*ro[1]

In [12]:
print "ranges would NOT set w[1] to zero :", (-ro[1]*2, ro[1]*2)
print "ranges would NOT set w[2] to zero:", (-ro[2]*2, ro[2]*2)

print "ranges of l1_penalty set w2 zero but NOT set w1 zero:", (-ro[1]*2, -ro[2]*2), (ro[2]*2, ro[1]*2)

ranges would NOT set w[1] to zero : (-175878940, 175878940)
ranges would NOT set w[2] to zero: (-161933396, 161933396)
ranges of l1_penalty set w2 zero but NOT set w1 zero: (-175878940, -161933396) (161933396, 175878940)



**The range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate
**

For W[1] to be zero, we need ro[1] in [-lambda/2, lambda/2]

We have -lambda/2 <= ro[1] <= lambda/2

This translates to lambda <= -2ro[1] and lambda >= 2ro[1]

For both conditions to be satisfied, lambda >= 2ro[1] = 1.75e8

we are looking for a value that will set both w1 and w0 to zero,

i.e. a lambda value that is greater than both 2ro[1] and 2ro[2] (in this case just 2*ro[1] as this is the greater)

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Implement coordinate descent that minimizes the cost function over a single feature i by using the formula above. Here the intercept (weight 0) is not regularized. The function accepts feature matrix, output, current weights, l1 penalty, and index of feature to optimize over, and returns new weight for feature i.

In [13]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:,i] * (output - prediction + weights[i]*feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2) 
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As we loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.


In [14]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    weights = initial_weights
    max_coordinate_changes = 0.0
    
    for i in range(len(weights)):
        old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
        # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
        #     and old values for weight[i], ..., weight[d-1]
        weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

        # use old_weights_i to compute change in coordinate
        coordinate_changes_i = old_weights_i - weights[i]
        if coordinate_changes_i > max_coordinate_changes:
            max_coordinate_changes = coordinate_changes_i
            # print max_coordinate_changes

    
    if max_coordinate_changes <= tolerance:
        return weights
    
    else:
        lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance)
           
    
            

**Using the following parameters, learn the weights on the sales dataset. **

In [15]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

** First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.**

In [16]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

__Then, run implementation of LASSO coordinate descent:__

In [23]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [24]:
lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

array([ 21624996.11991366,  63157248.84047247,         0.        ])

In [25]:
print weights

[ 21624996.11991366  63157248.84047247         0.        ]


**the RSS of the learned model on the normalized dataset **

In [26]:
predictions = predict_output(normalized_simple_feature_matrix, weights)
print "RSS of the learned model on the normalized dataset:{:.5e}".format( ((output - predictions)**2).sum())

RSS of the learned model on the normalized dataset:1.63049e+15


# Evaluating LASSO fit with more features

__Split the sales dataset into training and test sets.__

In [27]:
train_data,test_data = sales.random_split(.8,seed=0)

__Let us consider the following set of features.__

In [28]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

__First, create a normalized feature matrix from the TRAINING data with these features.  __

In [29]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

__First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  __

In [30]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0

In [33]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [34]:
print weights1e7

[ 24429599.89016737         0.                 0.          48389175.02950662
         0.                 0.           3317511.16366192
   7329961.94051543         0.                 0.                 0.
         0.                 0.                 0.        ]


*** Features had non-zero weight in this case ***

In [35]:
feature_list = ['constant'] + all_features

In [36]:
for k,v in zip(feature_list, weights1e7):
    if v != 0.0:
        print k, v

constant 24429599.8902
sqft_living 48389175.0295
waterfront 3317511.16366
view 7329961.94052


__Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  __

In [37]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0

In [39]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***Features had non-zero weight in this case***

In [40]:
for k,v in zip(feature_list, weights1e8):
    if v != 0.0:
        print k, v

constant 71114625.7528


__Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.__

In [41]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5

In [44]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

*** Features had non-zero weight in this case***

In [45]:
weights1e4

array([ 76259298.99182403, -22370834.16840227,  18573078.36061361,
        81736690.36290801,  -2145049.58385188, -10188516.60493549,
         6492635.81786232,   7189407.99249122,   1319350.29358088,
        12362479.75987415,  -5610729.08012918,  -3371915.78488671,
       -75364444.70568328,   2730228.52966835])

In [46]:
for k,v in zip(feature_list, weights1e4):
    if v != 0.0:
        print k, v

constant 76259298.9918
bedrooms -22370834.1684
bathrooms 18573078.3606
sqft_living 81736690.3629
sqft_lot -2145049.58385
floors -10188516.6049
waterfront 6492635.81786
view 7189407.99249
condition 1319350.29358
grade 12362479.7599
sqft_above -5610729.08013
sqft_basement -3371915.78489
yr_built -75364444.7057
yr_renovated 2730228.52967


## Rescaling learned weights

### Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [47]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [48]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print normalized_weights1e7[3]

161.317458506


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [49]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [50]:
prediction = predict_output(test_feature_matrix, normalized_weights1e4)
rss = ((test_output - prediction)**2).sum()
print 'RSS for model with weights1e4 is {:.5e}'.format(rss)

RSS for model with weights1e4 is 2.31426e+14


In [51]:
prediction = predict_output(test_feature_matrix, normalized_weights1e7)
rss = ((test_output - prediction)**2).sum()
print 'RSS for model with weights1e7 is {:.5e}'.format(rss)

RSS for model with weights1e7 is 2.75962e+14


In [52]:
prediction = predict_output(test_feature_matrix, normalized_weights1e8)
rss = ((test_output - prediction)**2).sum()
print 'RSS for model with weights1e8 is {:.5e}'.format(rss)

RSS for model with weights1e8 is 5.37166e+14


** By comparing the rss on test data of learned models, model with `l1_penalty = 1e4` performed best on the test data**